In [ ]:
import torch
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch import nn

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Load the Iris dataset
iris = load_iris()
X = torch.tensor(iris.data, dtype=torch.float32).to(device)
y = torch.tensor(iris.target, dtype=torch.long).to(device)  

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
class IrisClassifier(nn.Module):
  def __init__(self):
    super(IrisClassifier, self).__init__()
    self.fc1 = nn.Linear(4, 10)  # Input layer to first hidden layer
    self.relu = nn.ReLU()  # Activation function
    self.fc2 = nn.Linear(10, 10)  # First hidden layer to second hidden layer
    self.output = nn.Linear(10, 3)  # Second hidden layer to output layer

  def forward(self, x):
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))
    x = torch.nn.functional.softmax(self.output(x), dim=1)  # Apply softmax for probability distribution
    return x

In [ ]:
# Create the model instance and move it to the device
model = IrisClassifier().to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

# Define training loop
epochs = 100
for epoch in range(epochs):
  # Forward pass
  y_pred = model(X_train)
  loss = loss_fn(y_pred, y_train)

  # Backward pass and parameter update
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  # Print training progress
  if epoch % 10 == 0:
    print(f"Epoch: {epoch+1}, Loss: {loss.item():.4f}")

In [ ]:
# Evaluate the model on test data (already on device)
with torch.no_grad():
  y_pred = model(X_test)
  _, predicted = torch.max(y_pred, 1)
  accuracy = (predicted == y_test).sum().float() / len(y_test)
  print(f"Test Accuracy: {accuracy:.4f}")